<h2>Toronto Open Street Maps Data</h2>

For this project I will be analyzing the Open Street Maps data pertaining to the city of Toronto in Canada as it my city of residence.

- https://www.openstreetmap.org/relation/324211
- http://overpass-api.de/api/map?bbox=-79.6694,43.5635,-79.0851,43.8736

The original OSM XML file was exported from Overpass API using the following coordinates as the bounding area: -79.6694 to -79.0891 and 43.5635 to 43.8736.

The file was 495MB in size, as seen by the following terminal output:

<h3>Data Wrangling</h3>

In order to translate the xml file into a database, I first needed to take a look at the different nodes, 
node tags, etc. to get a  better sense of the data and see if anything needed to be fixed.

To do that, I first created a sample file of the xml data that contained every 100th top level element. This 
helped me quickly scan through the data to find the most recurring errors. The code for this can be found in 
the "1. make_sample_file.py" file. Once I had written the scripts and audited the sample file, I transitioned to the original xml file.

Next, I parsed through the xml file to identify the number of nodes, ways, etc. in the file. The code for this can be found in "2. count_tags.py". The output of the code for the original xml file is as follows:

I then audited the sample file to search for inconsistencies in the street names, postal codes, and city names. This was an iterative process that involved identifying the different groups of street name endings, types of postal code recordings, and city names, and then creating a list of acceptable types to filter out any inconsistencies.

These lists were then used later on with the original xml file to identify further inconsistencies and determine what changes needed to be made. The code for this can be found in "3. audit_addresses.py".

Some of the errors I noticed were:
- Variations of abbrevations/capitalizations for Street, Drive, Place, East, West, etc. (st., Pl., E., west)
- Spelling mistakes (Terace instead of Terrace)
- Complete addresses instead of just street names. This often included suite or floor numbers as well. (14th Avenue, Markham, Ont.; Lawrence Avenue West, 1st Floor)
- Postal codes with a space missing between the 3rd and 4th character (M9W1J8 vs. M9W 1J8)
- Postal codes with the last 3 characters missing (M9C)
- Postal codes in the wrong case (m1g2l6)
- Inconsistent city or town names (City of Brampton vs Brampton)
- Incorrect city names (Torontoitalian)

Once these errors were identified, I used the script "4. update_osm.py" to go through the xml file and make changes to fix the errors, mainly using dictionaries to map the incorrect entities to the correct ones. The corrections were then written to a new xml file. This had to be repeated a few times to catch stacked errors. 

For errors where the information was missing, such as with the truncated postal codes, I replaced the postal codes with 'Wrong Postal Code' to make it easy to identify later so the entry can be removed.

I also noticed that the province was listed under the key 'state' or 'province', and changed the keys to all be 'province' as is the terminology in Canada.

Once I was happy with the updated xml file, I used the script "5. convert_osm_to_csv.py" to convert the updated xml file to individual csv files for each table based on the schema provided. This resulted in 5 separate csv files that I could then import into the database. The files and their respective sizes (in MB) are as follows:

<h3>Database Creation</h3>

To create the database 'Toronto.db', I used sqlite3 in the command line to create the tables according to the 
schema provided and import the csv files. The commands used are listed in the text file 'sqlite3_commands.txt'.

The size of the database is 281MB as seen below:

<h3>Updating the Database</h3>

Now that the data was audited and imported, I could query it to find some interesting information about the city of Toronto. But before that, I needed to finish cleaning up the incorrect postal data. The tables 'ways_tags' and 'nodes_tags' contain the key 'postcodes' with the value 'Wrong Postal Code'. Using the following queries I identified the incorrect postal data and removed the values in the table:

<h3>Database Queries</h3>

Now that the database had been cleaned up I moved on to querying. First I wanted to verify that the number of nodes and ways aligned with the values obtained from the python script "2. count_tags.py".

- Number of nodes:

- Number of ways:

I then wanted to find the number of unique users that had contributed to this portion of the open street map, along with the top 5 contributing users:

- Number of unique users:  

- Top 5 contributing users:

I then wanted to dig deeper into the different keys in the nodes_tags table, and identify the top keys and explore those further.

- Top 10 keys in nodes_tags:    

- Top 10 amenities in Toronto:  

- Top 5 most popular restaurants:  

- Top 5 most popular cuisines:

From the queries above we can see that coffee shops top the list of cuisines in Toronto. Given that Tim Hortons is known as Canada's go-to coffee shop, whereas in America, Starbucks is infamously known to be on every city corner, I wanted to dig into the actual popularity of different coffee shops in Toronto. As coffee shops are listed as restaurants but also under cafes, I need to explore both types of amenities to verify their populatiry.

- Top 5 most popular coffee shops (restaurants):

- Top 5 most popular coffee shops (cafes):

In both cases, there are more Tim Hortons outlets than Starbucks in Toronto.

<h3>Other ideas about the dataset</h3>

In retrospect, having wrangled the xml data and converted in into a database to query, there are a few things I realised along the way and would have changed. While I included a snippet in my python code to change province information (state --> province and all values --> ON), not all the keys and values got caught and I realised that a few values were missed upon querying the database. I proceeded to fix the values after that (shown below), but would try to have a more robust systematic cleanse of the data in the python code itself.

The queries below show the variations in province names in the nodes_tags table and the process of updating them to 'ON'.

When updating the province data for the 'ways_tags' table, I noticed that Florida was listed as a province. Upon further exploring the information associated with that ID I realised that the entry was for an automotive shop in Florida, USA and removed all the information associated with that ID as well. The queries for this, along with those used to update the remaining province variations are shown below:

The following queries were used to change the key from 'state' to 'province'. No 'state' keys existed in the 'ways_tags' table so only the 'nodes_tags' table needed to be updated.

To make the wrangling process for the province data even more robust, I would need to include the province key in the audit of the xml file. This would prevent overwriting incorrect data, such as the Florida entry. However, without writing code to specifically address the incorrect ids and remove them, I would still need to query the database for quick removal as shown above.